### Problem statement:
Gdoc: https://docs.google.com/document/d/1xYIV3xRNxTjUaw1fBkwB0ujJxUH6l7Gc1Mrskzny6JI/edit?usp=sharing

Trying to track ROI target by adjusting cpc - assume that RPC doesnt change w/ cpc
```
ROI_obs = RPC_obs / CPC_obs
ROI_target = RPC_obs / CPC_target
ROI_obs * CPC_obs = ROI_target * CPC_target
CPC_target = ROI_obs/ROI_target * CPC_obs = RPC_obs / ROI_target
```
Currently we calculate `RPC_obs` against real data - but monietization data may be spares - or u may have to go so far in the past that it no longer apllies to present

Can use lead score as way to leverage more plentiful session data - w/o relying on monetization events

Can split `RPC_obs` into 2 factors and a bias term: `RPQ_obs`, `QPS_obs`, and `R0` (revenue at 0 quality).  Quality is an abstract unit - the idea is that is is derived from lead score and roughly linear w/ revenue on a session level basis.  Ideally - we want to be able to calculate quality w/in sql and aggregate it at query time so we dont have to work w/ session level data.  `QPS_obs` can be aggregated w/in a modifier bucket - using plentiful session data.  `RPQ_obs` can be aggregated accross the dataset - under the assumption that `RPQ_obs` will be stable accross an entire channel/platform/product - and possibly b/w platforms/products
```
RPC_obs = (RPQ_obs * QPS_obs + R0)
CPC_target = RPC_obs / ROI_target = (RPQ_obs * QPS_obs + R0) / ROI_target
```

### Domain
Biddable Dimensions:
1. Location (DMA)
2. Operating System
3. Device Type
4. Publisher (website ad was served on)
5. Time of Day

Split will most likely be on `(DMA,OS,Device)`.  Unsure if there will be enough session level data to support splitting on dimensions beyond that.  We might have to consider some split variables in isolation.  Alternately - we could try using a k-means or percentile-based clustering method to group our data w/in a very fine grained split.

### Evaluation
Bascially we want this `rps` estimation to capture long term trends w/o short term  noise.  Ideas:
1. plot out rolling 1,7,14,30,60,90 day estimates of rps using regular aggregations and using the lead score stabilized rps estimate
    - want time plots and histograms
1. compare the rps esimation to rolling rps w/ large window
    - at that pt - why not use that rolling rps to estimate?
1. if the idea is that we are compensating for data deficiency - why dont we use large buckets to measure bias - and small buckets to measure variance - than combine into some metric?

### Initial approach
Will just naively use `lead_score` as a standin for quality - I think that ultimately precision at that lead score would be the best quality metric - since it basically corresponds to observed conversion rate.  But I think that is best accomplished on @sperks side.  Lets see how this goes first.

For a given split - I will pull in revenue averages, bucket size, and lead score averages for each bucket.  Will compute `RPQ_obs` and `R0` by fitting a 1 var regressor w/ `y=rps_avg` and `X=[lead_score_avg]` - will use `bucket size` as sample weight.

NOTE: i guess there are actually 2 differnt model scores per lead - I will pivot those out and the linear regressor will have 2 weights - 1 for each model score

In [1]:
import sys
import re
import os
# detect if we are running from a `notebooks/*_shared` folder
# re.match("not.*shared",sys.path[0])
if sys.path[0].endswith("_shared"):
    sys.path[0] = "/".join(sys.path[0].split("/")[:-2])
assert sys.path[0].endswith("adtech")

from utils.env import load_env_from_aws
load_env_from_aws()


from ds_utils.db.connectors import HealthcareDW
from notebooks.aduriseti_shared.utils import *

import functools
import datetime
TABOOLA = "TABOOLA"
MEDIA_ALPHA = "MEDIAALPHA"
BING = "BING"
U65 = "HEALTH"
O65 = 'MEDICARE'

NOW = datetime.datetime.now()
DAY = datetime.timedelta(days=1)

start_date = NOW-30*DAY
end_date = NOW
# start_date = NOW-60*DAY
# end_date = NOW-30*DAY
product=None
traffic_source = None

Attempting to load environ from: `SM_ENV_BASE`
...Success!!


In [9]:
from models.data.queries.lead_score import \
    refresh_session_scores, \
    SCHEMA as DS_SCHEMA, \
    SCORE_TABLE
# df = refresh_session_scores(NOW-360*DAY,NOW)
# df

with HealthcareDW() as db:
    df = db.to_df(f"select * from {DS_SCHEMA}.{SCORE_TABLE} limit 100")
df

INFO:root:query: select * from data_science.materialized_session_scores limit... executed in 0.28 seconds


,topic,content_type,exchange,received,session_id,computed_dt,jornayaid,score,model,score_null,score_adv,score_supp
0,healthcare.lead.score.scored,,kraken,1616887693,20210327232359.b616d7aafd3e,2021-03-27,82BF6C48-BDB8-D88B-1FDA-B2C1EB1661D0,0.0062,None,0.0062,None,None
1,healthcare.lead.score.scored,,kraken,1616887304,20210327231749.d543f78c2924,2021-03-27,45C48738-2942-94C6-25C2-279135E66BCD,0.0000,None,0.0000,None,None
2,healthcare.lead.score.scored,,kraken,1616887464,20210327232230.7adf12f20f1f,2021-03-27,16A6C145-382A-D17C-4619-330AE5675D12,0.0057,None,0.0057,None,None
3,healthcare.lead.score.scored,,kraken,1616887759,20210327232800.eb951da20188,2021-03-27,56C53E30-6075-6AA4-F1E2-3AEA4EA055FE,0.0105,None,0.0105,None,None
4,healthcare.lead.score.scored,,kraken,1616887492,20210327232227.370cfd9e3368,2021-03-27,62E8B754-726A-518E-B7B7-81B4D98A8BCD,0.0200,None,0.0200,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
95,healthcare.lead.score.scored,,kraken,1607364710,20201207181023.29c0e99802c7,2020-12-07,,0.1196,None,0.1196,None,None
96,healthcare.lead.score.scored,,kraken,1607364949,20201207181244.c4c9c7e67eea,2020-12-07,,0.0904,None,0.0904,None,None
97,healthcare.lead.score.scored,,kraken,1607364861,20201207181318.c2066edd27c7,2020-12-07,,0.0452,None,0.0452,None,None
98,healthcare.lead.score.scored,,kraken,1607364110,20201207175655.f91419da4241,2020-12-07,,0.0660,None,0.0660,None,None


In [12]:
with HealthcareDW() as db:
    df = db.to_df(f"""select VERSION()""")
df

INFO:root:query: select VERSION()... executed in 0.17 seconds


,version
0,"PostgreSQL 8.0.2 on i686-pc-linux-gnu, compile..."


In [10]:
with HealthcareDW() as db:
    df = db.to_df(f"""
       SELECT  
           session_id,
           COUNT(*),
           MAX(computed_dt)
        
        
    """)

topic           healthcare.lead.score.scoredhealthcare.lead.sc...
content_type                                                     
exchange        krakenkrakenkrakenkrakenkrakenkrakenkrakenkrak...
received                                             160973867501
session_id      20210327232359.b616d7aafd3e20210327231749.d543...
jornayaid       82BF6C48-BDB8-D88B-1FDA-B2C1EB1661D045C48738-2...
score                                                      4.7719
model                                                           0
score_null                                                 4.7719
score_adv                                                       0
score_supp                                                      0
dtype: object

In [19]:

product_filter = "" if product is None else \
    f"AND UPPER(s.product) = UPPER('{product}')"
traffic_filter = "" if traffic_source is None else \
    f"AND UPPER(traffic_source) = UPPER('{traffic_source}')"
session_revenue_sql = f"""
SELECT
    session_id,
    sum(revenue) AS revenue
FROM tron.session_revenue
WHERE session_creation_date::DATE BETWEEN '{start_date}' AND '{end_date}'
    {product_filter}
    {traffic_filter}
GROUP BY 1
"""
geoip_sql = f"""
SELECT 
    l.*,
    b.netowrk_index,
    b.start_int,
    b.end_int
FROM 
    data_science.maxmind_ipv4_geo_blocks AS b
    JOIN data_science.maxmind_geo_locations AS l
        ON b.maxmind_id = l.maxmind_id
"""
lead_score_sql = f"""
SELECT
    r.topic,
    r.content_type,
    r.exchange,
    r.received,
    r.body.sessionid                        AS session_id,
    TO_DATE(r.body."on", 'YYYY-MM-DD')      AS computed_dt,
    r.body.jornayaid,
    r.body.score                            AS score,
    r.body.response.meta.model              AS model,
    CASE 
        WHEN model IS NULL THEN score 
        ELSE null
    END                                     AS score_null,
    CASE 
        WHEN model = 'med-adv' THEN score 
        ELSE null   
    END                                     AS score_adv,
    CASE 
        WHEN model = 'med-supp' THEN score 
        ELSE null   
    END                                     AS score_supp
FROM dl_landing.internal_kraken_leadscore_scored AS r
WHERE
    /* Data partitioned on date - these filters greatly speed query */
    (r.year > {start_date.year} OR 
        (r.year = {start_date.year} AND r.month >= {start_date.month})) 
    AND
    (r.year < {end_date.year} OR 
        (r.year = {end_date.year} AND r.month <= {end_date.month}))
"""
sql_query = f"""
WITH
    rps AS ({session_revenue_sql}),
    ip_locs AS ({geoip_sql}),
    lead_scores AS ({session_revenue_sql}),
    rps_tz_adj AS (
        SELECT
            s.session_id,
            s.creation_date                                         AS utc_ts,
            extract(
                HOUR FROM
                convert_timezone('UTC', l.time_zone, s.creation_date) 
                    - s.creation_date
            )::INT                                                  AS utc_offset,
            l.time_zone,
            convert_timezone('UTC', l.time_zone, s.creation_date)   AS user_ts,
            date_part(DOW, user_ts)::INT                            AS dayofweek,
            date_part(HOUR, user_ts) +
            CASE 
                WHEN date_part(MINUTE, user_ts)::INT BETWEEN 0 AND 14 THEN 0.0
                WHEN date_part(MINUTE, user_ts)::INT BETWEEN 15 AND 29 THEN 0.25
                WHEN date_part(MINUTE, user_ts)::INT BETWEEN 30 AND 44 THEN 0.5
                WHEN date_part(MINUTE, user_ts)::INT BETWEEN 45 AND 59 THEN 0.75
            END                                                     AS hourofday,
            l.subdivision_1_iso_code                                AS state,
            l.metro_code                                            AS dma,
            r.revenue
        FROM 
            tracking.session_detail AS s
            JOIN ip_locs as l
                ON ip_index(s.ip_address) = l.netowrk_index
                AND inet_aton(s.ip_address) BETWEEN l.start_int AND l.end_int
                AND l.country_iso_code = 'US'
            INNER JOIN rps as r
                ON s.session_id = r.session_id
            LEFT JOIN lead_scores as scores
                ON s.session_id = scores.session_id
        WHERE nullif(s.ip_address, '') IS NOT null
            AND nullif(dma,'') IS NOT NULL 
            AND s.creation_date::DATE BETWEEN '{start_date}' AND '{end_date}'
            {product_filter}
            {traffic_filter}
    )
SELECT
    *
FROM 
    rps_tz_adj
LIMIT 100
;
"""
with HealthcareDW() as db:
    session_rps_df = db.to_df(sql_query)
session_rps_df


INFO:root:query: with rps as ( SELECT session_id, sum(revenue) AS revenue FRO... executed in 33.87 seconds


,session_id,utc_ts,utc_offset,time_zone,user_ts,dayofweek,hourofday,state,dma,revenue
0,20210427041839.ee319fd40bcb,2021-04-27 04:18:39,-7,America/Los_Angeles,2021-04-26 21:18:39,1,21.25,WA,819,15.88
1,20210427041307.37630a90196a,2021-04-27 04:13:07,-5,America/Chicago,2021-04-26 23:13:07,1,23.00,WI,669,1.91
2,20210428022910.69d570c22f2e,2021-04-28 02:29:10,-5,America/Chicago,2021-04-27 21:29:10,2,21.25,MN,613,3.09
3,20210427184724.bcea86a22070,2021-04-27 18:47:24,-4,America/New_York,2021-04-27 14:47:24,2,14.75,FL,539,15.94
4,20210427201519.d1e77bba7a9c,2021-04-27 20:15:19,-4,America/New_York,2021-04-27 16:15:19,2,16.25,MD,512,10.72
...,...,...,...,...,...,...,...,...,...,...
95,20210427220919.99e594c17260,2021-04-27 22:09:19,-7,America/Los_Angeles,2021-04-27 15:09:19,2,15.00,CA,802,3.84
96,20210428031224.37284ad3405d,2021-04-28 03:12:24,-7,America/Los_Angeles,2021-04-27 20:12:24,2,20.00,CA,825,2.61
97,20210427141155.d758f1b01f77,2021-04-27 14:11:55,-5,America/Chicago,2021-04-27 09:11:55,2,9.00,MN,613,3.14
98,20210427222246.b06003c1cdc8,2021-04-27 22:22:46,-6,America/Boise,2021-04-27 16:22:46,2,16.25,ID,757,1.53


In [ ]:

product_filter = "" if product is None else \
    f"AND UPPER(s.product) = UPPER('{product}')"
traffic_filter = "" if traffic_source is None else \
    f"AND UPPER(traffic_source) = UPPER('{traffic_source}')"
session_revenue_sql = f"""
SELECT
    session_id,
    sum(revenue) AS revenue
FROM tron.session_revenue
WHERE session_creation_date::DATE BETWEEN '{start_date}' AND '{end_date}'
    {product_filter}
    {traffic_filter}
GROUP BY 1
"""
geoip_sql = f"""
SELECT 
    l.*,
    b.netowrk_index,
    b.start_int,
    b.end_int
FROM 
    data_science.maxmind_ipv4_geo_blocks AS b
    JOIN data_science.maxmind_geo_locations AS l
        ON b.maxmind_id = l.maxmind_id
"""
unified
agg_rps_query = f"""
with
    rps as ({session_revenue_sql}),
    ip_locs as ({geoip_sql}),
    rps_tz_adj as (
        SELECT
            s.creation_date                                         AS utc_ts,
            extract(
                HOUR FROM
                convert_timezone('UTC', l.time_zone, s.creation_date) 
                    - s.creation_date
            )::INT                                                  AS utc_offset,
            l.time_zone,
            convert_timezone('UTC', l.time_zone, s.creation_date)   AS user_ts,
            date_part(DOW, user_ts)::INT                            AS dayofweek,
            date_part(HOUR, user_ts) +
            CASE 
                WHEN date_part(MINUTE, user_ts)::INT BETWEEN 0 AND 14 THEN 0.0
                WHEN date_part(MINUTE, user_ts)::INT BETWEEN 15 AND 29 THEN 0.25
                WHEN date_part(MINUTE, user_ts)::INT BETWEEN 30 AND 44 THEN 0.5
                WHEN date_part(MINUTE, user_ts)::INT BETWEEN 45 AND 59 THEN 0.75
            END                                                     AS hourofday,
            l.subdivision_1_iso_code                                AS state,
            l.metro_code                                            AS dma,
            r.revenue
        FROM 
            tracking.session_detail AS s
            JOIN ip_locs as l
                ON ip_index(s.ip_address) = l.netowrk_index
                AND inet_aton(s.ip_address) BETWEEN l.start_int AND l.end_int
                AND l.country_iso_code = 'US'
            INNER JOIN rps as r
                ON s.session_id = r.session_id
        WHERE nullif(s.ip_address, '') IS NOT null
            AND nullif(dma,'') IS NOT NULL 
            AND s.creation_date::DATE BETWEEN '{start_date}' AND '{end_date}'
            {product_filter}
            {traffic_filter}
    )
SELECT
    {','.join(agg_columns)},
    COUNT(session_id)                                                       AS sessions,
    SUM((revenue>0)::INT::FLOAT)                                            AS num_leads,
    AVG((revenue>0)::INT::FLOAT)                                            AS lps_avg,
    SUM(revenue) / CASE
        WHEN num_leads = 0 THEN 1
        ELSE num_leads
    END                                                                     AS rpl_avg,
    (SUM(revenue) / COUNT(DISTINCT session_id))::NUMERIC(8,4)               AS rps_,
    AVG(revenue)                                                            AS rps_avg,
    STDDEV(revenue)                                                         AS rps_std,
    VARIANCE(revenue)                                                       AS rps_var
FROM rps_tz_adj
GROUP BY {','.join(agg_columns)}
"""
# print(agg_rps_query)
# print(traffic_filter)
from ds_utils.db.connectors import HealthcareDW
with HealthcareDW() as db:
    df = db.to_df(agg_rps_query)
globals()["df"] = df

delt = df["rps_avg"] - df['rps_']
if not all(delt.abs() < 1e-3):
    print("session uniqueness assummption not satisfied")
df = df \
    .sort_values(by=agg_columns, ascending=True) \
    .set_index(agg_columns)

df['int_ix'] = range(len(df))


In [ ]:
import datetime
TABOOLA = "TABOOLA"
MEDIA_ALPHA = "MEDIAALPHA"
BING = "BING"
U65 = "HEALTH"
O65 = 'MEDICARE'

NOW = datetime.datetime.now()
DAY = datetime.timedelta(days=1)

import functools
@functools.lru_cache()
def agg_rps(start_date,end_date,product,traffic_source,agg_columns):
    agg_columns = list(agg_columns)
    product_filter = "" if product is None else \
        f"AND UPPER(s.product) = UPPER('{product}')"
    traffic_filter = "" if traffic_source is None else \
        f"AND UPPER(traffic_source) = UPPER('{traffic_source}')"
    agg_rps_query = f"""
    with
        rps as (
            SELECT
                session_id,
                sum(revenue) AS revenue
            FROM tron.session_revenue
            WHERE session_creation_date::DATE BETWEEN '{start_date}' AND '{end_date}'
                {product_filter}
                {traffic_filter}
            GROUP BY 1
        ),
        ip_locs as (
            SELECT 
                l.*,
                b.netowrk_index,
                b.start_int,
                b.end_int
            FROM 
                data_science.maxmind_ipv4_geo_blocks AS b
                JOIN data_science.maxmind_geo_locations AS l
                    ON b.maxmind_id = l.maxmind_id
        ),
        rps_tz_adj as (
            SELECT
                s.*,
                s.creation_date                                         AS utc_ts,
                extract(
                    HOUR FROM
                    convert_timezone('UTC', l.time_zone, s.creation_date) 
                        - s.creation_date
                )::INT                                                  AS utc_offset,
                l.time_zone,
                convert_timezone('UTC', l.time_zone, s.creation_date)   AS user_ts,
                date_part(DOW, user_ts)::INT                            AS dayofweek,
                date_part(HOUR, user_ts) +
                CASE 
                    WHEN date_part(MINUTE, user_ts)::INT BETWEEN 0 AND 14 THEN 0.0
                    WHEN date_part(MINUTE, user_ts)::INT BETWEEN 15 AND 29 THEN 0.25
                    WHEN date_part(MINUTE, user_ts)::INT BETWEEN 30 AND 44 THEN 0.5
                    WHEN date_part(MINUTE, user_ts)::INT BETWEEN 45 AND 59 THEN 0.75
                END                                                     AS hourofday,
                l.subdivision_1_iso_code                                AS state,
                l.metro_code                                            AS dma,
                r.revenue
            FROM 
                tracking.session_detail AS s
                JOIN ip_locs as l
                    ON ip_index(s.ip_address) = l.netowrk_index
                    AND inet_aton(s.ip_address) BETWEEN l.start_int AND l.end_int
                    AND l.country_iso_code = 'US'
                INNER JOIN rps as r
                    ON s.session_id = r.session_id
            WHERE nullif(s.ip_address, '') IS NOT null
                AND nullif(dma,'') IS NOT NULL 
                AND s.creation_date::DATE BETWEEN '{start_date}' AND '{end_date}'
                {product_filter}
                {traffic_filter}
        )
    SELECT
        {','.join(agg_columns)},
        COUNT(session_id)                                                       AS sessions,
        SUM((revenue>0)::INT::FLOAT)                                            AS num_leads,
        AVG((revenue>0)::INT::FLOAT)                                            AS lps_avg,
        SUM(revenue) / CASE
            WHEN num_leads = 0 THEN 1
            ELSE num_leads
        END                                                                     AS rpl_avg,
        (SUM(revenue) / COUNT(DISTINCT session_id))::NUMERIC(8,4)               AS rps_,
        AVG(revenue)                                                            AS rps_avg,
        STDDEV(revenue)                                                         AS rps_std,
        VARIANCE(revenue)                                                       AS rps_var
    FROM rps_tz_adj
    GROUP BY {','.join(agg_columns)}
    """
    # print(agg_rps_query)
    # print(traffic_filter)
    from ds_utils.db.connectors import HealthcareDW
    with HealthcareDW() as db:
        df = db.to_df(agg_rps_query)
    globals()["df"] = df

    delt = df["rps_avg"] - df['rps_']
    if not all(delt.abs() < 1e-3):
        print("session uniqueness assummption not satisfied")
    df = df \
        .sort_values(by=agg_columns, ascending=True) \
        .set_index(agg_columns)

    df['int_ix'] = range(len(df))

    return df

campaign_split_fields = dict(
    # traffic_source = ["traffic_source"],
    browser = ["browser"],
    operating_system = ["operating_system"],
    device = ["device"],
    # channel = ["channel"],
    # domain = ["domain"],
    product = ["product"],
    # keyword = ["keyword"],
    # campaign_id = ["campaign_id"],
    # landing_page = ["landing_page"],
    TOD = ["dayofweek","hourofday"],
    dma = ["dma"],
    state =["state",],
    location = ["state","dma"],
    
    dma_os=["dma", "operating_system"],
    dma_device=["dma", "device", ],
    dma_os_device=["dma", "operating_system", "device"],

    state_os=["state", "operating_system"],
    state_device=["state", "device", ],
    state_os_device=["state", "operating_system", "device"],

    location_os = ["state", "dma", "operating_system"],
    location_device=["state", "dma", "device", ],
    location_os_device = ["state", "dma", "operating_system","device"],
)

taboola_val_map = {
    "device": {
        'DESKTOP': 'DESK',
        'MOBILE': 'PHON',
        'TABLET': 'TBLT',
    },
    "operating_system": {
        'Linux armv7l': "Linux",
        'Linux armv8l': "Linux",
        'Linux x86_64': "Linux",
        'MacIntel': 'Mac OS X',
        'Win32': "Windows",
        'iPad': "iPadOS",
        'iPhone': "iOS",
        '': None,
        'ARM': None,
        'Android': 'Android',
        'Linux aarch64': "Linux",
        'Win64': "Windows",
        'Linux armv7': "Linux",
        'Linux i686': "Linux",
        'Windows': "Windows",
    }
}

def translate_taboola_vals(df):
    index_cols = df.index.names
    df = df.reset_index()
    for c in df.columns:
        if c in taboola_val_map:
            df[c] = df[c].apply(taboola_val_map[c].__getitem__)
    df_bkp = df
    df = df \
        .groupby(index_cols) \
        .agg({
            "sessions": sum,
            "num_leads": sum,
            "lps_avg": get_wavg_by(df,"sessions"),
            "rpl_avg": get_wavg_by(df,"sessions"),
            "rps_avg": get_wavg_by(df,"sessions"),
        })
    df["int_ix"] = range(len(df))
    df_bkp_wavg = wavg(df_bkp[["lps_avg","rpl_avg","rps_avg"]],
                        df_bkp["sessions"].values.reshape(-1, 1))
    df_wavg = wavg(df[["lps_avg","rpl_avg","rps_avg"]],
                    df["sessions"].values.reshape(-1, 1))
    assert all((df_bkp_wavg - df_wavg).abs() < 1e-2), (df_bkp_wavg,df_wavg)
    return df

import pprint
from IPython.display import display as ipydisp    
import pandas as pd
from models.utils import wavg
import collections
import itertools

def wvar(V,W):
    mu = wavg(V,W)
    var = wavg((V - mu)**2,W)
    return var
def wstd(V,W):
    return wvar(V,W)**0.5
def get_wavg_by(df, col):
    def wavg_by(V):
        return wavg(V, W=df.loc[V.index, col])
    return wavg_by

def get_wthresh(W,p):
    W = rps_df["sessions"].sort_values(ascending=False)
    Wsum = W.sum()
    cumsum = 0
    for wthresh in W:
        if cumsum > Wsum * p:
            break
        cumsum += wthresh
    return wthresh

"""
TODO: 2021-05-20
  - test fitting on general traffic sources?
  - test clustering on multiple days of rps data - 
      or rolling rps data or something
  - try out more granular aggs
  - is there some kind of metric which measures total variance
      and computes how much of that variance is captured by a split?

Trevor: 2021-05-21
TODO:
- figure out how to minimize campaign # when writing back to taboola
    - want to make sure campaigns have sufficient traffic
- kw=(location,os,device)
- calc 30-day rps/kw
- can create distribution over the rps(kw) distribution
- 100 campaigns - 1 per percentile

TODO: 2021-05-24
- look into clustering each individual split variable
"""

# split2aggrps = {}
# for split,split_cols in campaign_split_fields.items():
#     print(split,split_cols)
#     rps_df = agg_rps(NOW-90*DAY,NOW,None,traffic_source=TABOOLA,agg_columns=split_cols)
#     rps_df = translate_taboola_vals(rps_df)
#     rps_df["split_on"] = split
#     split2aggrps[split] = rps_df
#     print(split,rps_df.shape)


def agg_rps(start_date, end_date, product, traffic_source, agg_columns):


def agg_rps_taboola(start_date, end_date, product, traffic_source, agg_columns):
    rps_df = agg_rps(start_date,end_date,None,traffic_source=traffic_source,agg_columns=agg_columns)
    rps_df = translate_taboola_vals(rps_df)
    rps_df["split_on"] = split

split2fitaggrps = {}
for split,split_cols in campaign_split_fields.items():
    print(split,split_cols)
    rps_df = agg_rps(NOW-90*DAY,NOW-30*DAY,None,traffic_source=TABOOLA,agg_columns=tuple(split_cols))
    rps_df = translate_taboola_vals(rps_df)
    rps_df["split_on"] = split
    split2fitaggrps[split] = rps_df

    print(split,rps_df.shape)

split2evalaggrps = {}
for split,split_cols in campaign_split_fields.items():
    print(split,split_cols)
    rps_df = agg_rps(NOW-30*DAY,NOW,None,traffic_source=TABOOLA,agg_columns=tuple(split_cols))
    rps_df = translate_taboola_vals(rps_df)
    rps_df["split_on"] = split
    split2evalaggrps[split] = rps_df

    print(split,rps_df.shape)


In [ ]:

import sklearn.cluster
import numpy as np
import scipy.stats
from matplotlib import pyplot as plt
def cluster_split(split):
    rps_df_fit = split2fitaggrps[split]
    rps_df_eval = split2evalaggrps[split]
    split_size = rps_df_fit.__len__()
    MINCLUST = 8
    if split_size <= MINCLUST:
        rps_clust_df = rps_df_eval
        rps_df_fit["clust"] = 0
        rps_df_eval["clust"] = 0
    else:
        # nclust = MINCLUST
        nclust = max(MINCLUST, int(np.log(split_size)))
        # nclust = max(MINCLUST,int(split_size ** 0.5))
        # print("nclust", nclust, split_size, np.log(split_size))
        clusterer = sklearn.cluster.KMeans(n_clusters=nclust)
        rps_df_fit["clust"] = clusterer \
            .fit_predict(
                rps_df_fit["rps_avg"].values.reshape(-1, 1),
                sample_weight=rps_df_fit["sessions"])
        rps_df_eval["clust"] = clusterer \
            .predict(
                rps_df_eval["rps_avg"].values.reshape(-1, 1),
                sample_weight=rps_df_eval["sessions"])
        rps_clust_df = rps_df_eval \
            .groupby("clust") \
            .agg({"rps_avg": get_wavg_by(rps_df_eval, "sessions"), "sessions": sum})
    assert rps_clust_df["rps_avg"].max() <= rps_df_eval["rps_avg"].max()
    rps_wavg = wavg(rps_df_eval["rps_avg"], rps_df_eval["sessions"])
    rps_clust_wavg = wavg(rps_clust_df["rps_avg"], rps_clust_df["sessions"])
    assert abs(rps_wavg - rps_clust_wavg) < 1e-4, (rps_wavg, rps_clust_wavg)
    return rps_df_fit,rps_df_eval,rps_clust_df

def get_split_factor(rps_df):
    orig_index = rps_df.index.names
    split_attr2unique_vals = {index_col: rps_df.index.unique(
        index_col) for index_col in orig_index}
    _,new_index_order = zip(*sorted((V.__len__(),c) for c,V in split_attr2unique_vals.items()))
    nclusts = rps_df["clust"].unique().__len__()
    split_factor = nclusts * np.prod([1] + [split_attr2unique_vals[c].__len__() for c in new_index_order[:-1]])
    return split_factor

perfD = []
for split in campaign_split_fields.keys():
    rps_df_fit,rps_df_eval,rps_clust_df = cluster_split(split)
    perfd = {
        "split": split,
        "fit_shape": rps_df_fit.shape,
        "clust_shape": rps_clust_df.shape,
        # wavg(rps_df["rps_avg"],rps_df["sessions"]),
        "split_variance": wstd(rps_df_eval["rps_avg"], rps_df_eval["sessions"]),
        "cluster_variance": wstd(rps_clust_df["rps_avg"], rps_clust_df["sessions"]),
        # wstd(rps_df["rps_avg"],rps_df["sessions"])
        "split_factor": rps_df_fit.__len__(),
        "clustered_split_factor": get_split_factor(rps_df_fit),
    }
    perfD.append(perfd)
    pprint.pprint(perfd)
    ipydisp(rps_clust_df)

perfdf = pd.DataFrame(perfD)
ipydisp(perfdf)
#%%
# BEST_SPLIT = "location_os_device"
BEST_SPLIT = "dma_os_device"
# BEST_SPLIT = "dma_os"

rps_df_fit, rps_df_eval, rps_clust_df = cluster_split(BEST_SPLIT)
rps_df = rps_df_fit
get_split_factor(rps_df)
orig_index = rps_df.index.names
split_attr2unique_vals = {index_col: rps_df.index.unique(index_col) for index_col in orig_index}
_,new_index_order = zip(*sorted((V.__len__(),c) for c,V in split_attr2unique_vals.items()))
rps_df = rps_df .reset_index()
campaign_df = rps_df \
    .groupby([*new_index_order[:-1], "clust"]) \
    .agg({
        "sessions": sum,
        "rps_avg": get_wavg_by(rps_df,"sessions"),
        new_index_order[-1]: tuple
    })
assert campaign_df["sessions"].sum() == rps_df_fit["sessions"].sum()
camp_rps_wavg = wavg(campaign_df["rps_avg"],campaign_df["sessions"])
fit_rps_wavg = wavg(rps_df_fit["rps_avg"], rps_df_fit["sessions"])
assert abs(camp_rps_wavg - fit_rps_wavg) < 1e-5

excl_campaign_df = campaign_df.groupby([*new_index_order[:-1]]) \
    .agg({
        new_index_order[-1]: tuple
    })
def flatten(M):
    return tuple(el for r in M for el in r)
excl_campaign_df[new_index_order[-1]] = excl_campaign_df[new_index_order[-1]] .apply(flatten)

camps = []
for idx,r in campaign_df.iterrows():
    camp = {
        "sessions_60d": r["sessions"], 
        "rps_avg_60d": r["rps_avg"]
    }
    for field,val in zip(new_index_order[:-1],idx):
        camp[field] = {"includes": val}
    last_field = new_index_order[-1]
    camp[last_field] = {
        "includes": r[last_field]
    }
    camps.append(camp)

for idx, r in excl_campaign_df.iterrows():
    camp = {}
    for field, val in zip(new_index_order[:-1], idx):
        camp[field] = {"includes": val}
    last_field = new_index_order[-1]
    camp[last_field] = {
        "excludes": r[last_field]
    }
    camps.append(camp)

camp_df = pd.DataFrame(camps)
camp_df.to_csv("campaign_dump.csv")
#%%

import numpy as np
import pandas as pd
import scipy.stats
import sklearn.feature_selection
import sklearn.metrics
from matplotlib import pyplot as plt
import sklearn.preprocessing

requires_campaign_split = [
    # "browser",
    "operating_system",
    "device",
    # "channel", # not actually sure what this is
    # "domain",
    "product",
    # "landing_page",
    "location",
]

for split in requires_campaign_split:
    rps_df = split2aggrps[split] .reset_index()
    Xs = rps_df \
        .apply(lambda r: [r["int_ix"]]*int(r["sessions"]), axis=1)
    X = np.concatenate(Xs.values).reshape(-1,1)
    ys = rps_df \
        .apply(lambda r: [r["rps_avg"]]*int(r["sessions"]),axis=1)
    y = np.concatenate(ys.values)
    # print(y.min(),np.quantile(y, 0.5),y.max())
    y = y > y.mean()
    # y = np.concatenate(ys.values).reshape(-1,1)
    # y = sklearn.preprocessing.KBinsDiscretizer(n_bins=2,encode="ordinal") \
    #     .fit_transform(y).reshape(-1)
    mi = sklearn.feature_selection.mutual_info_regression(X,y,discrete_features=True)
    print(split,mi,y.mean(),rps_df.shape)
#%%
y
#%%
sklearn.metrics.mutual_info_score

#%%
import numpy as np
import pandas as pd
import scipy.stats
from matplotlib import pyplot as plt

requires_campaign_split = [
    "browser",
    "operating_system",
    "device",
    # "channel", # not actually sure what this is
    # "domain",
    "product",
    # "landing_page",
    "location",
]
combined_rps_df = pd.concat([df.reset_index() for df in split2aggrps.values()])
splitI = combined_rps_df["split_on"].isin(requires_campaign_split)
density = scipy.stats.gaussian_kde(
    dataset=combined_rps_df.loc[splitI,"rps_avg"],
    weights=combined_rps_df.loc[splitI, "sessions"],
    # bw_method="scott",
    # bw_method="silverman",
    # bw_method=0.1,
)
xs = np.linspace(0,2,100)
plt.plot(xs,density(xs))
#%%
for split in requires_campaign_split:
    splitI = combined_rps_df["split_on"] == split
    density = scipy.stats.gaussian_kde(
        dataset=combined_rps_df.loc[splitI, "rps_avg"],
        weights=combined_rps_df.loc[splitI, "sessions"])
    xs = np.linspace(0, 2, 100)
    plt.plot(xs, density(xs))
    plt.title(split)
    plt.show()
#%%
combined_rps_df.loc[splitI, ["rps_avg","sessions"]].apply(
        lambda r: pd.Series([r["rps_avg"]]*int(r["sessions"])),axis=1) \
    .unstack()
#%%
agg_rps = split2aggrps["TOD"]
Xy = agg_rps[["sessions","rps"]].reset_index()
Xy["i"] = range(len(Xy))
Xy
#%%
import sklearn.feature_selection
sklearn.feature_selection.mutual_info_regression(
    []
)

#%%
"""
- 
- overall goal:
    - specific ROI targetting w/ minimal campaigns
- what this means for accnt structure
    - for many variables we must split campaigns to target ROI
    - want to capture greatest amt of rps variation w/ 
        fewest # of campaigns
- 2 ways of approaching this
    1. minimize rps variation w/in a campaign
        - i.e. after campaign split want to minimize rps variance w/in campaigns
        => i actually think this is eq to decision tree regression w/ split criterion MSE
        https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
        - tried this out a little - but the computation was intensive which made it slow going
    2. maximize rps variation outside campaigns
    - i.e. after campaign split want to minimize rps variance w/in campaigns
        => i actually think this is eq to decision tree regression w/ split criterion MSE
        https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
    - mostly focused on this approach

- approaches I tried
1. rank splits by their correlation/covaraince/MI w/ `agg_rps`
    - tried ANOVA,pt.biserial,MI - had most success w/ MI
    - was dificult to compute b/c couldnt find methods that accepted sample weight
    - MI approach was promising but didnt go down that route
2. rank splits by intra-split agg_rps variance
    - had most success w/ this
    - tested out an approahc where I cluster the split on rps - had good results
    - think this is the mtd to use going forward
    TODO:
    - test fitting on general traffic sources?
    - test clustering on multiple days of rps data - 
        or rolling rps data or something
    - try out more granular aggs
    - is there some kind of metric which measures total variance
        and computes how much of that variance is captured by a split?

3. fit decision tree on rps data 
    - MSE criterion is apparently the same as minimizng inter split variance
    - computationally intensive
    - not 100% clear how to go from tree to campaign structure


- can do this by
    1. choosing what vars or tuples of vars to split campaigns on
    2. grouping similar buckets w/in those splits
- want to measure correlation/dependence of categorical split vars w/ cont rps
    - cat,cat metrics w/ binned rps
        chi
    - cont,cont metrcs w/ 1 hot encoded split vars
    - cat,cont metrics
- correlation metrics
    - MI:
        - sklearn.feature_selection.mi_regression
        - would need AFAICT to rresample input arrays
    - ANOVA:
        - scipy.stats.f_oneway
    - pt biserial
        - needs binary vars tho
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pointbiserialr.html
        - https://towardsdatascience.com/point-biserial-correlation-with-python-f7cd591bd3b1
- want most variation of rps w/ fewest camapaigns
- dont want to group similar buckets together - think its too complicated

"""
#%%
requires_campaign_split = [
    "browser",
    "operating_system",
    "device",
    # "channel", # not actually sure what this is
    "domain",
    "product",
    "landing_page",
    "location",
]
#%%

traffic_source = TABOOLA
with HealthcareDW() as db:
    traffic_filter = "" if traffic_source is None else \
        f"AND UPPER(traffic_source) = UPPER('{traffic_source}')"
    sql = f"""
    select
        traffic_source,domain,count(*)
    from tracking.session_detail
    where True 
    {traffic_filter}
    group by traffic_source,domain;
    """
    df = db.to_df(sql).sort_values("count")
df
#%%

# %%
with HealthcareDW() as db:


In [ ]:
# Header

